In [41]:
import pandas as pd
import math

In [42]:
csv = "data/new_data.csv"
df = pd.read_csv(csv)
df = df.drop_duplicates()


In [43]:
# Функция для вычисления расстояния между координатами
def get_distance(lat1, long1, lat2, long2):
    R = 6373
    lat1 = math.radians(lat1)
    long1 = math.radians(long1)
    lat2 = math.radians(lat2)
    long2 = math.radians(long2)
    dlong = long2 - long1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlong / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c
    

In [44]:
# Функция для вычисления длины всего маршрута
def get_route_distance(id, df):
    distance = 0
    rows = df.loc[df['id'] == id]
    for i in range(1, len(rows)):
        distance += get_distance(rows['latitude'].iloc[i-1], rows['longitude'].iloc[i-1], rows['latitude'].iloc[i], rows['longitude'].iloc[i])
    return distance

In [45]:
# Функция для получения доли участков, где самолёт менял высоту
def get_altitude_distance(id, df):
    alt_inaccuracy = 67 # погрешность в изменении высоты, использовал таблицу:
    # http://hist.rloc.ru/lobanov/6_13.htm
    rows = df.loc[df['id'] == id]
    point_amount = len(rows)
    if point_amount <= 1:
        return 0
    id_distance = get_route_distance(id, df)
    if id_distance == 0:
        return 0
    altitudes = rows['altitude']
    latitudes = rows['latitude']
    longitudes = rows['longitude']
    alt_delta_dist = 0
    for i in range(1, point_amount):
        delta_alt = abs(altitudes.iloc[i] - altitudes.iloc[i-1])
        if delta_alt <= 120:
            alt_inaccuracy = 10
        if delta_alt < alt_inaccuracy:
            alt_delta_dist += get_distance(latitudes.iloc[i-1], longitudes.iloc[i-1], latitudes.iloc[i], longitudes.iloc[i])
    return alt_delta_dist

In [46]:
# Функция для получения доли участков, где самолёт менял направление
def get_heading_distance(id, df):
    heading_inaccuracy = 1 # погрешность в изменении направления
    rows = df.loc[df['id'] == id]
    point_amount = len(rows)
    if point_amount <= 1:
        return 0
    id_distance = get_route_distance(id, df)
    if id_distance == 0:
        return 0
    headings = rows['heading']
    latitudes = rows['latitude']
    longitudes = rows['longitude']
    head_delta_dist = 0
    for i in range(1, point_amount):
        if abs(headings.iloc[i] - headings.iloc[i-1]) < heading_inaccuracy:
            head_delta_dist += get_distance(latitudes.iloc[i-1], longitudes.iloc[i-1], latitudes.iloc[i], longitudes.iloc[i])
    return head_delta_dist

In [47]:
# Функция для получения доли участков, где самолёт менял высоту и направление
def get_heading_altitude_distance(id, df):
    rows = df.loc[df['id'] == id]
    point_amount = len(rows)
    if point_amount <= 1:
        return 0
    id_distance = get_route_distance(id, df)
    if id_distance == 0:
        return 0
    altitudes = rows['altitude']
    headings = rows['heading']
    latitudes = rows['latitude']
    longitudes = rows['longitude']
    head_alt_delta_dist = 0
    for i in range(1, point_amount):
        if abs(headings.iloc[i] - headings.iloc[i-1]) < heading_inaccuracy \
            and abs(altitudes.iloc[i] - altitudes.iloc[i-1]) < alt_inaccuracy:
            head_alt_delta_dist += get_distance(latitudes.iloc[i-1], longitudes.iloc[i-1], latitudes.iloc[i], longitudes.iloc[i])
    return head_alt_delta_dist

In [48]:
ids = {id: df['id'].unique() for id in df['id']}
whole_distance = 0
alt_change_distance = 0
head_change_distance = 0
head_alt_change_distance = 0
for id in ids:
    whole_distance += get_route_distance(id, df)
    alt_change_distance += get_altitude_distance(id, df)
    head_change_distance += get_heading_distance(id, df)
    head_alt_change_distance += get_heading_altitude_distance(id, df)


In [49]:
print("Доля маршрута полёта без изменения высоты")
print(alt_change_distance/whole_distance)
print("Доля маршрута полёта без поворота")
print(head_change_distance/whole_distance)
print("Доля маршрута полёта без поворота и без снижения высоты")
print(head_alt_change_distance/whole_distance)


Доля маршрута полёта без изменения высоты
0.6585814239677948
Доля маршрута полёта без поворота
0.6816482215225852
Доля маршрута полёта без поворота и без снижения высоты
0.5638734739544684
